# Fault Detection Model Trial Notebook

In [1]:
import sys
import os
import torch

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

## Load data

In [6]:
from data.config import DataConfig
from data.load import load_spring_particle_data

data_config = DataConfig()
data_config.set_train_dataset()

# get node and edge dataset path from which data will be loaded
node_ds_paths, edge_ds_paths = data_config.get_dataset_paths()

# load datalaoders
train_loader, valid_loader, test_loader, data_stats = load_spring_particle_data(node_ds_paths, edge_ds_paths)

ModuleNotFoundError: No module named 'data.load'

### Process input data

In [ ]:
dataiter = iter(train_loader)
data = next(dataiter)

n_nodes = data[0].shape[1]
n_timesteps = data[0].shape[2]
n_dims = data[0].shape[3]

print(f"Number of nodes: {n_nodes}")
print(f"Number of timesteps: {n_timesteps}")  
print(f"Number of dimensions: {n_dims}")

Number of nodes: 5
Number of timesteps: 49
Number of dimensions: 4


## Load model

In [2]:
from config import TrainAnomalyDetectornConfig
from anomaly_detection import AnomalyDetector

fdet_config = TrainAnomalyDetectornConfig()

In [ ]:
anomaly_detector = AnomalyDetector(anom_config=fdet_config.anom_config)
anomaly_detector.set_run_params(data_stats=data_stats,
                                domain=fdet_config.domain, 
                                norm_type=fdet_config.norm_type,)
# print model info
print("Anomaly Detector Model Initialized with the following configurations:")
anomaly_detector.print_model_info()

NameError: name 'data_stats' is not defined

## Train Model

In [ ]:
from anomaly_detection import TrainerAnomalyDetector
from torch.utils.tensorboard import SummaryWriter

train_log_path = fdet_config.get_train_log_path(n_nodes, n_timesteps, n_dims)

# initialize logger
if fdet_config.is_log:
    fdet_config.save_params()
    logger = SummaryWriter(log_dir=train_log_path)
else:
    logger = None

# initialize trainer
trainer = TrainerAnomalyDetector(log_path=train_log_path, logger=logger)
trainer.fit(anomaly_detector, train_loader)

NameError: name 'n_nodes' is not defined

## Test

In [ ]:
from config import get_model_pickle_path

trained_anomaly_detector = AnomalyDetector.load_from_pickle(get_model_pickle_path(train_log_path))



## Other stuff

In [1]:
import sys
import os
import torch

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import os
from data.config import DataConfig
from fault_detection.config import TrainAnomalyDetectornConfig

fdet_config = TrainAnomalyDetectornConfig()

data_config = DataConfig()
data_config.set_train_dataset()

log_path = fdet_config.get_train_log_path(500, 1)
print(log_path)

if log_path is not None:
    os.makedirs(log_path, exist_ok=True)

c:\Aryan_Savant\Thesis_Projects\my_work\AFD_implement_thesis\fault_detection\logs\bearing\cwru\scene_1\node=gearbox\healthy_unhealthy\0_N[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-007[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-021[]\anom=IF\T500_measures=[acc]\domain=time\fex=[first_n_modes]\anom(comps)=500\v1


In [ ]:
fdet_config.check_if_version_exists()

Version 1 already exists in the log path 'logs\spring_particles\P005\scenario_1\node=all_nodes\healthy\H1_[OG]\anom=SVM\T50_measures=[vel+pos]\domain=time\fex=[first_n_modes]\anom(comps)=980\v1'.


In [3]:
from fault_detection.config import SelectFaultDetectionModel

model_selector = SelectFaultDetectionModel()
model_selector.select_model_and_params()

bearing
└── cwru
    └── scene_1
        ├── <node_name>
        └── node=gearbox
            ├── <ds_type>
            └── healthy_unhealthy
                ├── <ds_subtype>
                └── 0_N[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-007[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-021[]
                    ├── <model>
                    └── anom=IF
                        ├── <ds_stats>
                        └── T500_measures=[acc]
                            ├── <domain>
                            └── domain=time
                                ├── <fex_type>
                                └── fex=[first_n_modes]
                                    ├── <shape_compatibility>
                                    └── anom(comps)=500
                                        ├── <version>
                                        └── v1 [0]


Available version paths:
0: logs/bearing\cwru\scene_1\node=gearbox\healthy_unhealthy\0_N[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-007[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-021[]\anom=IF\T500_measures=[acc]\domain=time\fex=[first_n_modes]\anom(comps)=500\v1


ValueError: invalid literal for int() with base 10: ''